In [87]:
# Install project dependencies
!pip install torch
!pip install transformers
!pip install sagemaker huggingface
!git lfs install
#!pip install bitsandbytes
#!pip install accelerate
!sudo yum update -y 
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs git -y

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


In [88]:
# INIT SAKEMAKER

import sagemaker

# init session to connect to other aws ressource
sess = sagemaker.Session()

# create session bucket to store project artefacts
sagemaker_session_bucket = "sagemaker-bert2bert"
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# IAM role to allow connection to session bucket and create model deployment
role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::159645953966:role/service-role/SageMaker-MLOps
sagemaker bucket: sagemaker-bert2bert
sagemaker session region: eu-central-1


In [120]:
!sudo yum update -y
!sudo yum install amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum update -y
!sudo yum install git-lfs -y

Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock
amzn2-core                                               | 3.6 kB     00:00     
amzn2extra-docker                                        | 2.9 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-livepatch                                     | 2.9 kB     00:00     
amzn2extra-python3.8                                     | 2.9 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
github_git-lfs/x86_64/signature                          |  819 B     00:00     
github_git-lfs/x86_64/signature                          |  951 B     00:02

  _  kernel-5.4               available    [ =stable ]
 50  selinux-ng               available    [ =stable ]
 52  tomcat9                  available    [ =stable ]
 53  unbound1.13              available    [ =stable ]
 54 †mariadb10.5              available    [ =stable ]
 55  kernel-5.10=latest       enabled      [ =stable ]
 56  redis6                   available    [ =stable ]
 57 †ruby3.0                  available    [ =stable ]
 58 †postgresql12             available    [ =stable ]
 59 †postgresql13             available    [ =stable ]
 60  mock2                    available    [ =stable ]
 61  dnsmasq2.85              available    [ =stable ]
 62  kernel-5.15              available    [ =stable ]
 63 †postgresql14             available    [ =stable ]
 64  firefox                  available    [ =stable ]
 65  lustre                   available    [ =stable ]
 66 †php8.1                   available    [ =stable ]
 67  awscli1                  available    [ =stable ]
 68 †php8.

In [55]:
#!git lfs install
#!rm -rf bert2bert_shared-german-finetuned-summarization
#!git clone https://huggingface.co/$repository

Cloning into 'bert2bert_shared-german-finetuned-summarization'...
remote: Enumerating objects: 28, done.
remote: Total 28 (delta 0), reused 0 (delta 0), pack-reused 28
Unpacking objects: 100% (28/28), 299.47 KiB | 1.58 MiB/s, done.


In [89]:
# DOWNLOAD MODEL  

import os
import tarfile
import torch
import boto3
from sagemaker.s3 import S3Uploader
from transformers import AutoModelForSeq2SeqLM, BertTokenizerFast 

# Define model to download
model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"

# Define s3 path to store the model
model_id = model_name.split("/")[-1]
s3_bucket = sess.default_bucket()
s3_path = f"s3://{s3_bucket}/origin"
s3_model_key = f"/origin/{model_id}.tar.gz"

s3_client = boto3.client('s3')

# check, if the model already exsists in the s3 bucket
try:
    response = s3_client.head_object(Bucket=s3_bucket, Key=s3_model_key)
    model_exists = True
    print("Model already exists in S3.")
except s3_client.exceptions.NoSuchKey:
    model_exists = False
    print(f"Model does not exist in S3 at {s3_model_key}. It will be uploaded.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '404':
        print(f"Model doesn't exist in S3 at {s3_model_key}. It will be uploaded.")
        model_exists = False
    else:
        raise


if not model_exists:
    # load model and tokenizer
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    
    # save artefacts temporary on local disk
    local_model_dir = f"./{model_id}"
    os.makedirs(local_model_dir, exist_ok=True)
    model.save_pretrained(local_model_dir)
    tokenizer.save_pretrained(local_model_dir)
    
    # create tar.gz
    tar_path = f"{model_id}.tar.gz"
    with tarfile.open(tar_path, "w:gz") as tar:
        tar.add(local_model_dir, arcname=os.path.basename(local_model_dir))
    
    # upload.tar.gz to s3
    s3_tar_path = f"{s3_path}/{tar_path}"
    s3_client.upload_file(tar_path, s3_bucket, f"{model_id}/{tar_path}")
    print(f"Model uploaded to S3: {s3_tar_path}")
    
    # clear local
    os.remove(tar_path)
    os.system(f"rm -rf {local_model_dir}")

Model doesn't exist in S3 at /origin/bert2bert_shared-german-finetuned-summarization.tar.gz. It will be uploaded.


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Model uploaded to S3: s3://sagemaker-bert2bert/origin/bert2bert_shared-german-finetuned-summarization.tar.gz


In [111]:
print(torch.cuda.is_available())

True


In [123]:
# MODEL QUANTIZATION COMPLETE

# these versions support 8-bit and 4-bit
!pip install bitsandbytes>=0.39.0 accelerate>=0.20.0


from transformers import AutoModelForSeq2SeqLM, BertTokenizerFast
import bitsandbytes as bnb


# Funktion, um das Modell lokal zu laden und zu quantisieren
def quantize_model(model_name, save_directory):
    model_name = 'mrm8488/bert2bert_shared-german-finetuned-summarization'
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    # Ersetze die linearen Schichten durch 8-Bit-quantisierte Versionen
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
    
    # Speichere das quantisierte Modell lokal
    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)

# Lokalen Pfad für das quantisierte Modell definieren
quant_model_dir = f"8-bit-quant-{model_id}"

# Quantisierung nur durchführen, wenn das quantisierte Modell nicht bereits im S3 existiert
try:
    s3.head_object(Bucket=sagemaker_session_bucket, Key=f"{quant_model_dir}/model.tar.gz")
    quant_model_exists = True
except:
    quant_model_exists = False

if not quant_model_exists:
    # Quantisiere und speichere das Modell lokal
    quantize_model(model_id, quant_model_dir)

    # Packe das quantisierte Modell als .tar.gz
    !tar -czvf 8-bit-quant-model.tar.gz -C {quant_model_dir} .
    
    # Pfad im S3 für das quantisierte Modell definieren
    s3_quant_model_path = f"s3://{sagemaker_session_bucket}/{quant_model_dir}/model.tar.gz"
    
    # Hochladen des gepackten quantisierten Modells in den S3-Bucket
    S3Uploader.upload('8-bit-quant-model.tar.gz', s3_quant_model_path)


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or `pip install bitsandbytes`.

In [124]:
!pip install bitsandbytes>=0.39.0 accelerate>=0.20.0
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert2bert_shared-german-finetuned-summarization", device_map="auto", torch_dtype=torch.bfloat16)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [2]:
# OPTIMZATION: Using BertTokenizerFast

from transformers import pipeline, BertTokenizer, BertTokenizerFast
import time
import os

# Define Model name
model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"

# Define Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name)
#tokenizer = BertTokenizer.from_pretrained(model_name)
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Init download
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer)

# start timer
start_time = time.time()

# Test-Input
text_input = (
   "Veränderungen und Innovationen sind allgegenwärtig, eine kontinuierliche und nachhaltige Weiterentwicklung der Mitarbeitenden ist von zentraler Bedeutung. Hier setzt das Praxis-Coaching an, denn es ist eine wirkungsvolle Methode, um individuelle Lern- und Entwicklungsprozesse zu fördern und zu gestalten. Das Praxis-Coaching Data Science unterstützt Sie dabei, den Praxistransfer in den Unternehmenskontext zu optimieren und die Rolle des Data Scientists in Ihrer Arbeitsumgebung zu etablieren. Es hilft Ihnen dabei, an die Themen des Seminars praxisnah anzuknüpfen, und ermöglicht im virtuellen 1:1-Raum mit dem:der Coach:in den Austausch für individuelle Impulse und Tipps. Der Ansatz dieses Coachings beinhaltet weder eine Wissensvermittlung noch eine Unternehmensberatung. Es geht hier in erster Linie darum, Ihre Rolle als Data Scientist gemeinsam mit dem:der Coach:in zu analysieren und Ihr Selbstvertrauen in diesem Bereich zu fördern. Das Praxis-Coaching Data Science adressiert genau dieses Dilemma: Es bringt Ihre Daten-Ideen und Ihr Wissen aus den Fachbereichen mit der Erfahrung unserer Expert:innen bei der erfolgreichen Implementierung von Datenprojekten zusammen. Egal ob eine oder zehn Stunden - buchen Sie über unser Anfrageformular ein flexibel nutzbares Kontingent mit unseren Trainer:innen. Preis pro Stunde: 250€ (297,50€ inkl MwSt)."
    "Geben Sie im nächsten Schritt bei „Anfragen“ im Bemerkungsfeld an, bei welchem:welcher Trainer:in Sie gerne das Coaching durchführen wollen und wie viele Stunden Sie buchen möchten. Der:Die Trainer:in meldet sich daraufhin bei Ihnen."
    "Inhalte"
    "Das Praxis-Coaching Data Science unterstützt Sie bei den folgenden Möglichkeiten:"
    "Entwicklung: Etablierung Ihrer Jobrolle als Data Scientist."
    "Follow-up: Anknüpfung an die Kursinhalte und Reflexion für den maximalen Praxistransfer."
    "Entfaltung: Individuelle Impulse und Tipps für die Umsetzung von Maßnahmen in Ihrem Businesskontext."
    "Ihr Nutzen"
    "Nutzen Sie dieses Praxis-Coaching für die Förderung und Gestaltung Ihres individuellen Lern- und Entwicklungsprozesses:"
    "Schaffen Sie im 1:1 mit dem Experten Klarheit für Ihre neue Rolle als Data Scientist und stärken Sie dieses Mindset für sich und Ihr Team."
)

results = summarizer(text_input)

#print summarizer results
for result in results:
    print(f"Zusammenfassung: {result['summary_text']}")

# end timer
end_time = time.time()

# print time
execution_time = end_time - start_time
print(f"Ausführungszeit: {execution_time} Sekunden")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Zusammenfassung: Veränderungen und Innovationen sind allgegenwärtig, eine kontinuierliche und nachhaltige Weiterentwicklung der Mitarbeitenden ist von zentraler Bedeutung. Hier setzt das Praxis - Coaching an, denn es ist eine wirkungsvolle Methode, um individuelle Lern - und Entwicklungsprozesse zu fördern und zu gestalten.
Ausführungszeit: 4.239929914474487 Sekunden


In [45]:
# Use DLC Container
# https://github.com/aws/deep-learning-containers

# Using own build container
https://github.com/aws/amazon-sagemaker-examples/tree/main/inference/torchserve/mme-gpu/workspace/docker

SyntaxError: invalid syntax (60234713.py, line 5)

In [ ]:
# Use Deepspeed
# https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

In [ ]:
# Inference Code from Huggingface transformer libary 
!pygmentize code/inference_code.py


https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/deploy-pretrained-model/BERT/Deploy_BERT.ipynb

In [ ]:
# Packaging the model

import tarfile

zipped_model_path = os.path.join(model_path, "model.tar.gz")

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add(model_path)
    tar.add(code_path)

In [ ]:
# Push model package to regitry

In [13]:
# Deploy Model


from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

endpoint_name = 'bert-base'

model = PyTorchModel(entry_point='inference_code.py', 
                     model_data=zipped_model_path, 
                     role=get_execution_role(), 
                     framework_version='1.5', 
                     py_version='py3')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.xlarge', 
                         endpoint_name=endpoint_name)

# Was ist der unerschied von model zu huggingface_estimator?
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

In [ ]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [ ]:
# Inference
# https://docs.aws.amazon.com/de_de/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html


import boto3

sm = boto3.client('sagemaker-runtime')

prompt = "The best part of Amazon SageMaker is that it makes machine learning easy."

response = sm.invoke_endpoint(EndpointName=endpoint_name, 
                              Body=prompt.encode(encoding='UTF-8'),
                              ContentType='text/csv')

response['Body'].read()



In [ ]:
# Serverless Inference
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

# https://github.com/aws/amazon-sagemaker-examples/blob/main/serverless-inference/huggingface-serverless-inference/huggingface-text-classification-serverless-inference.ipynb

In [ ]:
# Add metrics to scale based on traffic
# https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-inference-recommender/auto-scaling


In [ ]:
# HF model registration
# https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-inference-recommender/huggingface-inference-recommender/huggingface-inference-recommender.ipynb


In [14]:
# Clean up
llm.delete_model() # TODO
predictor.delete_endpoint()


NameError: name 'predictor' is not defined